In [1]:
%load_ext autoreload
%autoreload 2

from sklearn.pipeline import Pipeline

from src.preprocess import PipelinePreprocessor

In [5]:
pipeline = Pipeline([('preprocess', PipelinePreprocessor(generator=False))])

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [12]:
sentences = [
    "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
    "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
    "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/."
]

X = pipeline.fit_transform(sentences)
for x in X:
    print(' '.join(x), end='\n\n')

Preprocessing...: 100%|██████████| 3/3 [00:00<00:00, 620.46it/s]

<allcaps> cant wait </allcaps> for the new season of <hashtag> twin peaks </hashtag> <happy> ! <repeated> <hashtag> david lynch </hashtag> <hashtag> tv series </hashtag> <happy>

i saw the new <hashtag> john doe </hashtag> movie and it sucks <elongated> ! <repeated> <allcaps> waisted </allcaps> <money> . <repeated> <hashtag> bad movies </hashtag> 😕

<user> : can not wait for the <date> <hashtag> sentiment </hashtag> talks ! <allcaps> yay <elongated> </allcaps> ! <repeated> <laugh> <url>

